## Importing Libries

In [1]:
import numpy as numpy
import pandas as pd
import glob 

states=glob.glob('dataset/states*')

## Combine all data

In [2]:
states_df=[]

for state in states:
  data=pd.read_csv(state)
  states_df.append(data)
  
combined_states=pd.concat(states_df)

combined_states

,Unnamed: 0,State,TotalPop,Hispanic,White,Black,Native,Asian,Pacific,Income,GenderPop
0,0,Rhode Island,1053661,13.36%,74.33%,5.68%,0.35%,3.25%,0.04%,"$59,125.27",510388M_543273F
1,1,South Carolina,4777576,5.06%,62.89%,28.75%,0.29%,1.25%,0.05%,"$46,296.81",2322409M_2455167F
2,2,South Dakota,843190,3.24%,82.50%,1.42%,9.42%,1.02%,0.04%,"$51,805.41",423477M_419713F
3,3,Tennessee,6499615,4.72%,73.49%,18.28%,0.23%,1.41%,0.04%,"$47,328.08",3167756M_3331859F
4,4,Texas,26538614,38.05%,44.69%,11.65%,0.26%,3.67%,0.07%,"$55,874.52",13171316M_13367298F
5,5,Utah,2903379,13.47%,79.41%,1.02%,1.08%,2.20%,0.83%,"$63,488.92",1459229M_1444150F
0,0,Utah,2903379,13.47%,79.41%,1.02%,1.08%,2.20%,0.83%,"$63,488.92",1459229M_1444150F
1,1,Vermont,626604,1.61%,93.98%,0.98%,0.30%,1.24%,0.03%,"$55,602.97",308573M_318031F
2,2,Virginia,8256630,8.01%,63.27%,20.18%,0.21%,5.46%,0.06%,"$72,866.01",4060948M_4195682F
3,3,Washington,6985464,11.14%,72.04%,3.38%,1.41%,7.02%,0.61%,"$64,493.77",3487725M_3497739F


## MElt

In [3]:
reshaped_data=pd.melt(
    combined_states,
    id_vars=['State','TotalPop','Income','GenderPop'],
    value_vars=['Hispanic','White','Black','Native','Asian','Pacific'],
    var_name='race',
    value_name='PopPercentage'
)
reshaped_data

,State,TotalPop,Income,GenderPop,race,PopPercentage
0,Rhode Island,1053661,"$59,125.27",510388M_543273F,Hispanic,13.36%
1,South Carolina,4777576,"$46,296.81",2322409M_2455167F,Hispanic,5.06%
2,South Dakota,843190,"$51,805.41",423477M_419713F,Hispanic,3.24%
3,Tennessee,6499615,"$47,328.08",3167756M_3331859F,Hispanic,4.72%
4,Texas,26538614,"$55,874.52",13171316M_13367298F,Hispanic,38.05%
...,...,...,...,...,...,...
355,Alaska,733375,"$70,354.74",384160M_349215F,Pacific,1.06%
356,Arizona,6641928,"$54,207.82",3299088M_3342840F,Pacific,0.17%
357,Arkansas,2958208,"$41,935.63",1451913M_1506295F,Pacific,0.15%
358,California,38421464,"$67,264.78",19087135M_19334329F,Pacific,0.35%


## Checking For Duplicates

In [4]:

reshaped_data.duplicated().sum()

54

## droping duplicates

In [5]:
reshaped_data=reshaped_data.drop_duplicates()
reshaped_data.duplicated().sum()

0

In [6]:
reshaped_data.head()

,State,TotalPop,Income,GenderPop,race,PopPercentage
0,Rhode Island,1053661,"$59,125.27",510388M_543273F,Hispanic,13.36%
1,South Carolina,4777576,"$46,296.81",2322409M_2455167F,Hispanic,5.06%
2,South Dakota,843190,"$51,805.41",423477M_419713F,Hispanic,3.24%
3,Tennessee,6499615,"$47,328.08",3167756M_3331859F,Hispanic,4.72%
4,Texas,26538614,"$55,874.52",13171316M_13367298F,Hispanic,38.05%


## cleaning Income feature

In [7]:
reshaped_data.Income=reshaped_data.Income.str.replace('[$,]','',regex=True)
reshaped_data.Income

/var/folders/yh/7k5bf4ln10s4_h2kj1zxwpnh0000gn/T/ipykernel_56372/2500333235.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reshaped_data.Income=reshaped_data.Income.str.replace('[$,]','',regex=True)


0      59125.27 
1      46296.81 
2      51805.41 
3      47328.08 
4      55874.52 
         ...    
354    43296.36 
355    70354.74 
356    54207.82 
357    41935.63 
358    67264.78 
Name: Income, Length: 306, dtype: object

## cleaning and reshaping GenderPop feature

In [8]:
split_data=reshaped_data.GenderPop.str.split('_',expand=True)
split_data

,0,1
0,510388M,543273F
1,2322409M,2455167F
2,423477M,419713F
3,3167756M,3331859F
4,13171316M,13367298F
...,...,...
354,2341093M,2489527F
355,384160M,349215F
356,3299088M,3342840F
357,1451913M,1506295F


 ## cleaning PopPercentage

In [9]:
reshaped_data.PopPercentage=reshaped_data.PopPercentage.str.replace('(\%)','',regex=True)
reshaped_data.PopPercentage

<>:1: SyntaxWarning: invalid escape sequence '\%'
<>:1: SyntaxWarning: invalid escape sequence '\%'
/var/folders/yh/7k5bf4ln10s4_h2kj1zxwpnh0000gn/T/ipykernel_56372/4047955742.py:1: SyntaxWarning: invalid escape sequence '\%'
  reshaped_data.PopPercentage=reshaped_data.PopPercentage.str.replace('(\%)','',regex=True)
/var/folders/yh/7k5bf4ln10s4_h2kj1zxwpnh0000gn/T/ipykernel_56372/4047955742.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reshaped_data.PopPercentage=reshaped_data.PopPercentage.str.replace('(\%)','',regex=True)


0      13.36
1       5.06
2       3.24
3       4.72
4      38.05
       ...  
354     0.03
355     1.06
356     0.17
357     0.15
358     0.35
Name: PopPercentage, Length: 306, dtype: object

## getting male Population and changing data type


In [10]:
reshaped_data['malePop']=split_data[0].str.replace('M','',regex=True)
reshaped_data['malePop']=pd.to_numeric(reshaped_data['malePop'])

/var/folders/yh/7k5bf4ln10s4_h2kj1zxwpnh0000gn/T/ipykernel_56372/2687513442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reshaped_data['malePop']=split_data[0].str.replace('M','',regex=True)
/var/folders/yh/7k5bf4ln10s4_h2kj1zxwpnh0000gn/T/ipykernel_56372/2687513442.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reshaped_data['malePop']=pd.to_numeric(reshaped_data['malePop'])


## getting female Population and changing data type

In [11]:
reshaped_data['femalePop']=split_data[1].str.replace('F','',regex=True)
reshaped_data['femalePop']=pd.to_numeric(reshaped_data['femalePop'])

/var/folders/yh/7k5bf4ln10s4_h2kj1zxwpnh0000gn/T/ipykernel_56372/1720275793.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reshaped_data['femalePop']=split_data[1].str.replace('F','',regex=True)
/var/folders/yh/7k5bf4ln10s4_h2kj1zxwpnh0000gn/T/ipykernel_56372/1720275793.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reshaped_data['femalePop']=pd.to_numeric(reshaped_data['femalePop'])


In [12]:
reshaped_data=reshaped_data.drop(columns=['GenderPop'])
reshaped_data

,State,TotalPop,Income,race,PopPercentage,malePop,femalePop
0,Rhode Island,1053661,59125.27,Hispanic,13.36,510388,543273.0
1,South Carolina,4777576,46296.81,Hispanic,5.06,2322409,2455167.0
2,South Dakota,843190,51805.41,Hispanic,3.24,423477,419713.0
3,Tennessee,6499615,47328.08,Hispanic,4.72,3167756,3331859.0
4,Texas,26538614,55874.52,Hispanic,38.05,13171316,13367298.0
...,...,...,...,...,...,...,...
354,Alabama,4830620,43296.36,Pacific,0.03,2341093,2489527.0
355,Alaska,733375,70354.74,Pacific,1.06,384160,349215.0
356,Arizona,6641928,54207.82,Pacific,0.17,3299088,3342840.0
357,Arkansas,2958208,41935.63,Pacific,0.15,1451913,1506295.0


## NaN values

In [13]:
reshaped_data

,State,TotalPop,Income,race,PopPercentage,malePop,femalePop
0,Rhode Island,1053661,59125.27,Hispanic,13.36,510388,543273.0
1,South Carolina,4777576,46296.81,Hispanic,5.06,2322409,2455167.0
2,South Dakota,843190,51805.41,Hispanic,3.24,423477,419713.0
3,Tennessee,6499615,47328.08,Hispanic,4.72,3167756,3331859.0
4,Texas,26538614,55874.52,Hispanic,38.05,13171316,13367298.0
...,...,...,...,...,...,...,...
354,Alabama,4830620,43296.36,Pacific,0.03,2341093,2489527.0
355,Alaska,733375,70354.74,Pacific,1.06,384160,349215.0
356,Arizona,6641928,54207.82,Pacific,0.17,3299088,3342840.0
357,Arkansas,2958208,41935.63,Pacific,0.15,1451913,1506295.0


In [14]:
reshaped_data.isna().sum()

State             0
TotalPop          0
Income            0
race              0
PopPercentage     4
malePop           0
femalePop        12
dtype: int64